In [ ]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_double_side_view, show_selected_joints, get_fixed_boundary_joint)
import vis.fields
import matplotlib.cm as cm
import time

In [ ]:
# rod_length = curved_linkage.totalRestLength() / curved_linkage.numSegments()
rod_length = 0.079785061039255
width = rod_length / 25 * 5
thickness = width / 5 * 0.5
print(width, thickness)
scale = 1
io = InputOrganizer('lilium_{}'.format(scale), thickness, width, weaving_dir)

In [3]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 0.01
sw = 0.01

In [4]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = get_double_side_view(curved_linkage, flip = True)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…

In [5]:
curved_linkage.setExtendedDoFsPSRL(np.load('lilium_1_dof.npy'))

In [7]:
topColor =[180/255., 158/255., 119/255.]
bottomColor =[79/255., 158/255., 246/255.]
heights = curved_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [8]:
fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage)

In [9]:
curved_linkage.attraction_weight = 1e-5

In [10]:
def eqm_callback(prob, i):
    curved_linkage_view.update(scalarField = colors)

In [11]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = fixed_boundary_joints)

0	0.000502871	0.56214	0.56214	1	1
1	7.94156e-05	0.133527	0.133527	1	1
2	3.76452e-05	0.0304685	0.0304685	1	1
3	2.62905e-05	0.0131233	0.0131233	1	1
4	2.0414e-05	0.00713251	0.00713251	1	1
5	1.60493e-05	0.00389691	0.00389691	1	1
6	1.25827e-05	0.0020626	0.0020626	1	1
7	9.97326e-06	0.00107228	0.00107228	1	1
8	8.27265e-06	0.000562561	0.000562561	1	1
9	7.36279e-06	0.000319893	0.000319893	1	1
10	6.94733e-06	0.000208843	0.000208843	1	1
11	6.73359e-06	0.000158586	0.000158586	1	1
12	6.54904e-06	0.000132897	0.000132897	1	1
13	6.3174e-06	0.000113392	0.000113392	1	1
14	6.00973e-06	9.53508e-05	9.53508e-05	1	1
15	5.63031e-06	9.31441e-05	9.31441e-05	1	1
16	5.20849e-06	0.000151755	0.000151755	1	1
17	4.77729e-06	0.000266198	0.000266198	1	1
18	4.35932e-06	0.000333447	0.000333447	1	1
19	3.97459e-06	0.000349005	0.000349005	1	1
20	3.66635e-06	0.000321758	0.000321758	1	1
21	3.46745e-06	0.000218024	0.000218024	1	1
22	3.35545e-06	0.000136756	0.000136756	1	0
23	3.31474e-06	0.000426482	0.000426482	1	0
24	3.25951e-

In [12]:
def visualize_crossing(index, linkage, view):
    vf = show_selected_joints(linkage, [index], flip = True)
    view.update(vectorField = vf, scalarField = colors)

In [13]:
ribbons = order_segments_by_ribbons(curved_linkage)
_, _, _, _, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, curved_linkage, rest = True)
all_joint_index_list = [j_list + [j_list[0]] for j_list in all_joint_index]

def get_ribbon_crossing_list(index, linkage, view):
    selected_list = []
    selected_ribbon = []
    for ribbon_index, index_list in enumerate(all_joint_index_list):
        if index in set(index_list):
            selected_ribbon.append(ribbon_index)
            selected_list.append(index_list)
    print("The crossing {} belongs to ribbon {}".format(index, ', '.join([str(x) for x in selected_ribbon])))
    for i in range(len(selected_list)):
        print('Ribbon {}: {}'.format(selected_ribbon[i], selected_list[i]))

In [14]:
def select_crossing(crossing_index):
    visualize_crossing(crossing_index, curved_linkage, curved_linkage_view)
    get_ribbon_crossing_list(crossing_index, curved_linkage, curved_linkage_view)
select_crossing(59)

The crossing 59 belongs to ribbon 0, 25
Ribbon 0: [0, 1, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 50, 51, 48, 49, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 0, 0]
Ribbon 25: [59, 135, 100, 160, 383, 182, 424, 204, 423, 226, 422, 370, 421, 420, 419, 411, 418, 399, 417, 416, 415, 30, 59]


In [15]:
vf = show_selected_joints(curved_linkage, [61, 62, 63, 64, 65, 66, 67, 68, 12])
curved_linkage_view.update(vectorField = vf, scalarField = colors)